# 0. Информация об видеокарте.

In [1]:
!nvidia-smi

Tue Apr 16 12:13:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  | 00000000:01:00.0  On |                  N/A |
| 59%   49C    P5              50W / 370W |    906MiB / 10240MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Загурзка библиотек и объявление путей

In [1]:
import os
import glob
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import json
from pprint import pprint
import yaml
from ultralytics import YOLO
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

#### Список датасетов и названий директорий в директории DATASETS:

     СarDetection 
     https://www.kaggle.com/datasets/sshikamaru/car-object-detection
     
     CarsDetection 
     https://www.kaggle.com/datasets/abdallahwagih/cars-detection/data
     
     CarPerson
     https://www.kaggle.com/datasets/owaiskhan9654/car-person-v2-roboflow
     
     STN 
     https://github.com/andreluizbvs/PLAD/releases/download/1.0/plad.zip

     UtilityPole
     https://github.com/R3ab/ttpla_dataset?tab=readme-ov-file
     
     Transmission
     https://drive.usercontent.google.com/download?id=1Yz59yXCiPKS0_X4K3x9mW22NLnxjvrr0&export=download&authuser=0
     
     Tree
     https://universe.roboflow.com/tank-detect/tree-dataset-owarv
     
     TreeDetection 
     https://universe.roboflow.com/yolo-for-tree-detection/tree-detection-by-season-type
     
     Trees100
     https://universe.roboflow.com/dataset-x4ge5/dataset_trees_100_images
     
     BirdAnnotation
     https://universe.roboflow.com/drone-when8/bird-annotation-drp1n/dataset/1
     
     BirdsDetector
     https://universe.roboflow.com/jess-minaya/birds-detector-tis9s/dataset/1
     
     BirdsV3 
     https://universe.roboflow.com/kurisnis/birds-olyak/dataset/3
     
     BuildingDetection
     https://universe.roboflow.com/david-wardan/buildingdetection/dataset/1/images?split=test
     
     JoshEdits
     https://universe.roboflow.com/tour-de-chicago/josh-edits-2.5-outterclassifier

In [3]:
# Пути до датасетов с автомобилями
PATH_CAR = '../data/DATASET/CarDetection' # Путь до датасета Car Detection
PATH_CARS = '../data/DATASET/CarsDetection/Cars Detection/' # Путь до датасета Cars Detection
PATH_CAR_PERSON = '../data/DATASET/CarPerson/Car-Person-v2-Roboflow-Owais-Ahmad/' # Путь до датасета Car Person

# Пути до датасетов с ЛЭП
PATH_STN = '../data/DATASET/STN' # Путь до датасета STN PLAD
PATH_UTILLUTY_POLE = '../data/DATASET/UtilityPole/' # Путь до датасета Utiluty Pole
PATH_TRANSMISSION = '../data/DATASET/Transmission/data_original_size_v1/data_original_size/' # Путь до датасета Transmission

# Пути до датасетов с деревьями
PATH_TREE = '../data/DATASET/Tree/' # Путь до датасета Tree Detection
PATH_TREE_DETECTION = '../data/DATASET/TreeDetection/' # Путь до датасета Tree Detection
PATH_TREE_100 = '../data/DATASET/Trees100/' # Путь до датасета Tree 100

# Пути до датасетов с птицами
PATH_BIRD_ANNOTATION = '../data/DATASET/BirdAnnotation/' # Путь до датасета Bird Annotation 
PATH_BIRDS_DETECTOR = '../data/DATASET/BirdsDetector/' # Путь до датасета Birds Detector
PATH_BIRDS_V3 = '../data/DATASET/BirdsV3/' # Путь до датасета BirdsV3

# Пути до датасетов со зданиями
PATH_BUILDING_DETECTION = '../data/DATASET/BuildingDetection/' # Путь до датасета Building Detection
PATH_JOSH_EDITS = '../data/DATASET/JoshEdits/' # Путь до датасета JoshEdits

# Словарь по понижению размрености датасетов
DATASET_DOWNGRADES = {
    'dataCarDetection': 2, 
    'dataCarPerson': 3, 
    'dataCarsDetection': 4,
    'dataSTN': 1, 
    'dataUnilityPole': 1, 
    'dataTransmission': 1,
    'dataTree': 2, 
    'dataTreeDetection': 2, 
    'dataTree100': 2,
    'dataBirdAnnotation': 4, 
    'dataBirdsDetectod': 4, 
    'dataBirdsV3': 2,
    'dataBuildingDetection': 2, 
    'dataJoshEdits': 2
}

# 2. Обработка данных

In [4]:
def print_variable_name(variable):
    '''
        Функция позволяет получить имя переменной
        Input: 
            variable - переменная
        Output:
            variable_name - имя переменной
    '''
    variable_name = [name for name, value in globals().items() if value is variable][-1]
    return variable_name

In [5]:
def converterYoloToDataFrame(labels_list, path, classes):
    '''
        Функция позволяет перенести все данные с формата YOLO в DataFrame
        Input:
            labels_list - набор названий изображений
            path - путь до папки с изображениями
    '''
    labels = {} # Словарь объктов
    number_item = 0 # Номер записи
    for item in tqdm(labels_list):
        # Читаем все записи
        with open(path + item) as f:
            lines = f.readlines()
    
        # Проходимся по каждой записи
        for line in lines:
            # Получаем координаты в процентах
            x_center, y_center, w, h = (float(number) for number in line.split()[1:]) 
    
            # Заносим в словарь
            labels[number_item] = {
                'image_name': '/'.join(path.split('/')[:-2]) + '/images/' + item[:-3] + 'jpg',
                'x_center': x_center, 
                'y_center': y_center, 
                'w': w, 
                'h': h,
                'classes': classes
            }
            number_item += 1
    return labels

In [6]:
def datasetMinimization(dataSet, count):
    '''
        Функция позволяет понизить размер DataFrame за счёт удаления фотографий
        Input:
            dataSet - первоначальный DataFrame
            count - количество, во сколько раз сжать DataFrame
    '''
    # Список во всеми именами фотографий
    list_image_name = dataSet['image_name'].unique().tolist()

    # Кол-во фотографий в начале
    start_count_image = len(list_image_name)
    
    # Получаем именя изображений, которые оставляем
    name_file = random.sample(list_image_name, len(list_image_name) // count)

    # Записи с этими файлами
    dataSet = dataSet[dataSet['image_name'].isin(name_file)]

    # Кол-во фотографий после обработки
    end_count_image = len(dataSet['image_name'].unique())
    
    # Выведем системноо сообщение об результате сокращения фотографий
    print(f'There were {start_count_image} photos, now there are {end_count_image}')
    
    return dataSet

## 2.0 Классы для модели
    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

## 2.1. Датасеты с автомобилями
### 2.1.1 Обработка датасета Car Detection

In [7]:
img_h, img_w = (380, 676) # Размеры изображения
dataCarDetection = pd.read_csv(PATH_CAR + '/train_solution_bounding_boxes (1).csv') # Читаем датасет

dataCarDetection.rename(columns={'image':'image_name'}, inplace=True) # Переименовываем колонку

dataCarDetection.sample(10) # посмотрим на данные

,image_name,xmin,ymin,xmax,ymax
435,vid_4_6260.jpg,510.668596,185.567889,622.193922,224.203668
132,vid_4_16500.jpg,232.344428,193.392857,390.338640,254.036358
145,vid_4_17180.jpg,67.013025,200.728765,171.201158,239.364543
160,vid_4_17580.jpg,143.808973,196.816281,315.988423,256.481660
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
416,vid_4_6160.jpg,585.507959,169.428893,676.000000,211.977156
371,vid_4_29900.jpg,30.816208,200.239704,113.481910,228.116152
111,vid_4_14460.jpg,371.751085,177.253861,486.700434,226.159910
383,vid_4_29980.jpg,222.072359,198.283462,287.617945,224.203668
283,vid_4_22580.jpg,21.522431,199.261583,117.395080,239.853604


In [8]:
# Получим координаты центра объекта в процентах 
dataCarDetection['x_center'] = (dataCarDetection['xmin'] + dataCarDetection['xmax']) / 2 / img_w 
dataCarDetection['y_center'] = (dataCarDetection['ymin'] + dataCarDetection['ymax']) / 2 / img_h 

# Получим размеры объекта в процентах
dataCarDetection['w'] = (dataCarDetection['xmax'] - dataCarDetection['xmin']) / img_w # Получаем ширину объекта в процентах
dataCarDetection['h'] = (dataCarDetection['ymax'] - dataCarDetection['ymin']) / img_h # Получаем ширину объекта в процентах
dataCarDetection['classes'] = 0 # Задаём класс объекта
dataCarDetection.drop(
    ['xmin', 'xmax', 'ymin', 'ymax'], # Удаляем ненужные признаки
    axis=1, inplace=True
)

# Изменим путь до файла
dataCarDetection['image_name'] = f'{PATH_CAR}/training_images/' + dataCarDetection['image_name']

# Очистим память 
del img_h, img_w

# Уменьшаем размер DataFrame
dataCarDetection = datasetMinimization(dataCarDetection, DATASET_DOWNGRADES[print_variable_name(dataCarDetection)])

dataCarDetection.sample(10) # Посмотрим на данные

There were 355 photos, now there are 177


,image_name,x_center,y_center,w,h,classes
486,../data/DATASET/CarDetection/training_images/v...,0.805535,0.519956,0.134013,0.091969,0
342,../data/DATASET/CarDetection/training_images/v...,0.899421,0.514720,0.131693,0.096525,0
238,../data/DATASET/CarDetection/training_images/v...,0.904124,0.531451,0.191751,0.153153,0
5,../data/DATASET/CarDetection/training_images/v...,0.761577,0.517294,0.198987,0.189189,0
176,../data/DATASET/CarDetection/training_images/v...,0.620116,0.570061,0.254703,0.176319,0
453,../data/DATASET/CarDetection/training_images/v...,0.832127,0.528877,0.109986,0.088803,0
487,../data/DATASET/CarDetection/training_images/v...,0.585215,0.538091,0.108521,0.086788,0
449,../data/DATASET/CarDetection/training_images/v...,0.024240,0.569418,0.048480,0.082368,0
108,../data/DATASET/CarDetection/training_images/v...,0.960203,0.504424,0.079595,0.119691,0
215,../data/DATASET/CarDetection/training_images/v...,0.110709,0.582931,0.154848,0.093951,0


## 2.1.2. Обработка датасета Cars Detection

In [9]:
dataCarsDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CARS}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CARS}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarsDetection = pd.concat([dataCarsDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarsDetection = datasetMinimization(dataCarsDetection, DATASET_DOWNGRADES[print_variable_name(dataCarsDetection)])

dataCarsDetection.sample(10) # Посмотрим на данные

  0%|          | 0/878 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

There were 1254 photos, now there are 313


,image_name,x_center,y_center,w,h,classes
452,../data/DATASET/CarsDetection/Cars Detection/t...,0.301683,0.528846,0.438702,0.46875,0
628,../data/DATASET/CarsDetection/Cars Detection/t...,0.646635,0.419471,0.03125,0.014423,0
1023,../data/DATASET/CarsDetection/Cars Detection/t...,0.456731,0.52524,0.716346,0.65625,0
1505,../data/DATASET/CarsDetection/Cars Detection/t...,0.129808,0.722356,0.257212,0.401442,0
1354,../data/DATASET/CarsDetection/Cars Detection/t...,0.644231,0.615385,0.514423,0.63101,0
417,../data/DATASET/CarsDetection/Cars Detection/t...,0.942308,0.723558,0.063702,0.055288,0
399,../data/DATASET/CarsDetection/Cars Detection/v...,0.168269,0.723558,0.057692,0.034856,0
32,../data/DATASET/CarsDetection/Cars Detection/t...,0.84976,0.594952,0.233173,0.310096,0
168,../data/DATASET/CarsDetection/Cars Detection/t...,0.490385,0.542067,0.980769,0.633413,0
34,../data/DATASET/CarsDetection/Cars Detection/v...,0.669471,0.417067,0.204327,0.21875,0


### 2.1.3. Обработка датасета Car Person

In [10]:
dataCarPerson = pd.DataFrame(columns=dataCarDetection.columns)
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_CAR_PERSON}{sample}/labels') # Список файлов с разметкой
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_CAR_PERSON}{sample}/labels/', 0)) # Получаем датасет с разметкой
    dataCarPerson = pd.concat([dataCarPerson, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataCarPerson = datasetMinimization(dataCarPerson, DATASET_DOWNGRADES[print_variable_name(dataCarPerson)])

dataCarPerson.sample(10) # Посмотрим на данные

  0%|          | 0/1571 [00:00<?, ?it/s]

  0%|          | 0/448 [00:00<?, ?it/s]

  0%|          | 0/224 [00:00<?, ?it/s]

There were 2243 photos, now there are 747


,image_name,x_center,y_center,w,h,classes
1801,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.514423,0.294471,0.013221,0.049279,0
10499,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.762019,0.854567,0.106971,0.288462,0
1597,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.358173,0.536058,0.013221,0.018029,0
5596,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.768029,0.774038,0.205529,0.448317,0
572,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.909856,0.478365,0.03726,0.286058,0
842,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.241587,0.545673,0.013221,0.0625,0
2519,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.522837,0.495192,0.840144,0.399038,0
335,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.507212,0.354567,0.03125,0.085337,0
157,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.963942,0.478365,0.069712,0.063702,0
610,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.036058,0.957933,0.072115,0.080529,0


## 2.2. Обработка датасет с ЛЭП
### 2.2.1 Обработка датасета STN_PLAD

In [11]:
# Читаем файл с разметкой датасета STN
with open(PATH_STN + '/annotations.json') as f:
    dataSTN = json.load(f)

pprint(dataSTN['categories']) # Посмотрим на категории

# Получим два DataFrame, один с изображениями, другой с разметкой объектов
data = pd.DataFrame(dataSTN['images']).rename(columns={'id': 'image_id'})
data1 =  pd.DataFrame(dataSTN['annotations'])

# Объединим два дасатсета и удалим лишние признаки
dataSTN = pd.merge(data, data1, on='image_id', how='left').drop(
    [
        'licence', 'segmentation', 'date_captured', 'coco_url', 
        'iscrowd', 'image_id'
    ], axis=1
)

# Очистим память 
del data, data1 

dataSTN.sample(10) # Посмотрим на данные

[{'id': 0, 'name': 'tower', 'supercategory': 'tower'},
 {'id': 1, 'name': 'insulator', 'supercategory': 'component'},
 {'id': 2, 'name': 'spacer', 'supercategory': 'component'},
 {'id': 3, 'name': 'damper', 'supercategory': 'component'},
 {'id': 5, 'name': 'plate', 'supercategory': 'component'}]


,file_name,height,width,flickr_url,area,bbox,category_id,id
560,20181127-A1/DJI_0478.JPG,3078,5472,,81972,"[3365, 1698, 198, 414]",1,562
1898,20181127-A1/DJI_0517.JPG,3078,5472,,1496,"[2029, 1213, 68, 22]",3,1906
894,20181129-R/DJI_0017.JPG,3648,5472,,1380,"[3171, 3008, 69, 20]",3,898
104,20181127-A1/DJI_0460.JPG,3078,5472,,1725,"[2871, 2174, 69, 25]",3,105
787,20181127-A1/DJI_0500.JPG,3078,5472,,123715,"[1457, 1967, 227, 545]",1,791
82,20181127-A1/DJI_0479.JPG,3078,5472,,3648,"[2848, 2569, 96, 38]",3,83
1267,20181127-A1/DJI_0504.JPG,3078,5472,,117126,"[2817, 967, 482, 243]",1,1274
934,20181129-R/DJI_0019.JPG,3648,5472,,19872,"[808, 1391, 108, 184]",0,938
1597,20181129-R/DJI_0012.JPG,3648,5472,,1102,"[2455, 1778, 58, 19]",3,1604
1531,20181129-R/DJI_0061.JPG,3648,5472,,1919,"[1881, 2190, 101, 19]",3,1538


In [12]:
# Получим координаты объекта
dataSTN[['xmin', 'ymin', 'xmax', 'ymax']] = dataSTN['bbox'].apply(lambda x: pd.Series(x))
dataSTN['xmax'] = dataSTN['xmax'] + dataSTN['xmin']
dataSTN['ymax'] = dataSTN['ymax'] + dataSTN['ymin']

# Получим координаты центра объекта в процентах 
dataSTN['x_center'] = (dataSTN['xmin'] + dataSTN['xmax']) / 2 / dataSTN['width']
dataSTN['y_center'] = (dataSTN['ymin'] + dataSTN['ymax']) / 2 / dataSTN['height']

# Получим размеры объекта в процентах
dataSTN['w'] = (dataSTN['xmax'] - dataSTN['xmin']) / dataSTN['width']
dataSTN['h'] = (dataSTN['ymax'] - dataSTN['ymin']) / dataSTN['height']

# Переименуем признаки
dataSTN.rename(
    columns={
        'file_name': 'image_name',
        'category_id': 'classes'
    }, 
    inplace=True
) 

# Удалим ненужные признаки
dataSTN.drop(
    [
        'height', 'width', 'flickr_url',
        'area', 'bbox', 'id', 'xmin', 
        'xmax', 'ymin', 'ymax'
    ], 
    axis=1, inplace=True
)

# Изменим путь до файла
dataSTN['image_name'] = PATH_STN + '/' + dataSTN['image_name']

# Оставим фотографии принадлижащие только к опоре ЛЭП
dataSTN = dataSTN[dataSTN['classes'] == 0]
dataSTN['classes'] = dataSTN['classes'] + 1

dataSTN.sample(10) # Посомтрим на данные

,image_name,classes,x_center,y_center,w,h
1,../data/DATASET/STN/20181129-R/DJI_0021.JPG,1,0.293129,0.435855,0.014620,0.115132
1961,../data/DATASET/STN/20181129-R/DJI_0004.JPG,1,0.137061,0.175439,0.043860,0.101974
1660,../data/DATASET/STN/20181127-A1/DJI_0485.JPG,1,0.295230,0.155621,0.016630,0.062378
2142,../data/DATASET/STN/20181127-A1/DJI_0513.JPG,1,0.515716,0.596166,0.811404,0.682261
2194,../data/DATASET/STN/20181127-A1/DJI_0496.JPG,1,0.500000,0.492853,0.451023,0.641975
1307,../data/DATASET/STN/20181127-A1/DJI_0439.JPG,1,0.462537,0.561891,0.107822,0.876218
2039,../data/DATASET/STN/20181127-A1/DJI_0473.JPG,1,0.814510,0.362248,0.213085,0.723847
2101,../data/DATASET/STN/20181129-R/DJI_0006.JPG,1,0.503746,0.433388,0.724232,0.101425
1031,../data/DATASET/STN/20181127-A1/DJI_0417.JPG,1,0.700841,0.502437,0.376462,0.995127
760,../data/DATASET/STN/20181129-R/DJI_0001.JPG,1,0.504934,0.427357,0.036550,0.083333


### 2.2.2. Обработка датасета Utility Pole

In [13]:
dataUnilityPole = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'test']:
    labels_name = os.listdir(f'{PATH_UTILLUTY_POLE}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_UTILLUTY_POLE}{sample}/labels/', 1)) # Получаем датасет с разметкой
    dataUnilityPole = pd.concat([dataUnilityPole, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память
dataUnilityPole.sample(10) # Посмотрим на данные

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
129,../data/DATASET/UtilityPole/train/images/image...,0.36899,0.394231,0.362981,0.677885,1
41,../data/DATASET/UtilityPole/train/images/a93a3...,0.223558,0.629808,0.091346,0.307692,1
140,../data/DATASET/UtilityPole/train/images/image...,0.262019,0.719952,0.081731,0.34375,1
126,../data/DATASET/UtilityPole/train/images/image...,0.522837,0.519231,0.247596,0.831731,1
67,../data/DATASET/UtilityPole/train/images/bafa0...,0.294471,0.475962,0.28125,0.884615,1
170,../data/DATASET/UtilityPole/train/images/image...,0.882212,0.520433,0.105769,0.521635,1
263,../data/DATASET/UtilityPole/train/images/up3_j...,0.439904,0.527644,0.221154,0.824519,1
104,../data/DATASET/UtilityPole/train/images/falli...,0.489183,0.52524,0.266827,0.71875,1
212,../data/DATASET/UtilityPole/train/images/lws-e...,0.569712,0.518029,0.173077,0.709135,1
194,../data/DATASET/UtilityPole/train/images/istoc...,0.824519,0.75,0.052885,0.173077,1


### 2.2.3. Обработка датасета Transmission

In [14]:
list_image = glob.glob(PATH_TRANSMISSION + '*.jpg')
labels = {} # Словарь объктов
number_item = 0 # Номер записи
for path_image in tqdm(list_image):
    with open(path_image[:-3] + 'json') as f:
        json_file = json.load(f)
    xmax = 0
    xmin = 2**10
    ymax = 0
    ymin = 2**10
    height = json_file['imageHeight']
    width = json_file['imageWidth']
    for item in json_file['shapes']:
        if item['label'] in ['tower_wooden', 'cable']:
            for data in item['points']:
                x, y = data
                xmax = x if x > xmax else xmax
                ymax = y if x > ymax else ymax
                xmin = x if y < xmin else xmin
                ymin = y if y < ymin else ymin
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height
            labels[number_item] = {
                'image_name': path_image,
                'x_center': 0.0 if x_center < 0.0 else 1.0 if x_center > 1.0 else x_center, 
                'y_center': 0.0 if y_center < 0.0 else 1.0 if y_center > 1.0 else x_center, 
                'w': 0.0 if w < 0.0 else 1.0 if w > 1.0 else w, 
                'h': 0.0 if h < 0.0 else 1.0 if h > 1.0 else h,
                'classes': 1 if item['label'] == 'tower_wooden' else 5
            }
            number_item += 1
            
dataTransmission = pd.DataFrame(labels).T # Переносим все в DataFrame

# Очищаем память
del list_image, labels, number_item, xmin, xmax, ymin, ymax, h, w, json_file 

# Уменьшаем размер DataFrame
dataTransmission = datasetMinimization(dataTransmission, DATASET_DOWNGRADES[print_variable_name(dataTransmission)])

dataTransmission.sample(10) # Посмотрим на данные

  0%|          | 0/1242 [00:00<?, ?it/s]

There were 1231 photos, now there are 1231


,image_name,x_center,y_center,w,h,classes
7432,../data/DATASET/Transmission/data_original_siz...,0.682843,0.682843,0.248898,0.726496,5
3167,../data/DATASET/Transmission/data_original_siz...,0.512858,0.512858,0.972331,0.997396,5
9107,../data/DATASET/Transmission/data_original_siz...,0.521966,0.521966,0.725861,0.21118,5
8410,../data/DATASET/Transmission/data_original_siz...,0.537546,0.537546,0.623346,0.625655,5
7762,../data/DATASET/Transmission/data_original_siz...,0.824282,0.824282,0.277999,0.81679,5
1943,../data/DATASET/Transmission/data_original_siz...,0.78917,0.78917,0.419707,0.323866,5
1554,../data/DATASET/Transmission/data_original_siz...,0.499675,0.499675,0.99935,0.304151,5
6834,../data/DATASET/Transmission/data_original_siz...,0.421484,0.421484,0.073698,0.839744,5
7310,../data/DATASET/Transmission/data_original_siz...,0.49067,0.49067,0.279526,0.672182,5
4481,../data/DATASET/Transmission/data_original_siz...,0.705924,0.705924,0.587632,0.016087,5


## 2.3. Обработка датасетов с деревьями
### 2.3.1. Обработка датасета Tree Object Detecion

In [15]:
dataTree = pd.DataFrame(columns=dataCarDetection.columns)
labels_name = os.listdir(f'{PATH_TREE}train/labels/') # Список файлов с разметкой
labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE}train/labels/', 2)) # Получаем датасет с разметкой
dataTree = pd.concat([dataTree, labels.T]) # Соединяем выборки в единый датасет
    
del labels_name, labels # Очищаем память

dataTree.sample(10) # Посмотрим на данные

  0%|          | 0/214 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
427,../data/DATASET/Tree/train/images/JjyAZPIDj9IJ...,0.501563,0.392969,0.996875,0.645312,2
116,../data/DATASET/Tree/train/images/3888_jpg.rf....,0.260156,0.83125,0.057813,0.1,2
514,../data/DATASET/Tree/train/images/RGXRFD_jpg.r...,0.680469,0.714063,0.104688,0.13125,2
370,../data/DATASET/Tree/train/images/images-59-_j...,0.502344,0.496094,0.995313,0.976562,2
49,../data/DATASET/Tree/train/images/20230709_155...,0.550781,0.182031,0.401562,0.364063,2
313,../data/DATASET/Tree/train/images/images-46-_j...,0.107031,0.317188,0.214062,0.634375,2
91,../data/DATASET/Tree/train/images/2tnc_5652334...,0.389062,0.449219,0.404687,0.88125,2
121,../data/DATASET/Tree/train/images/3888_jpg.rf....,0.390625,0.842969,0.040625,0.0875,2
444,../data/DATASET/Tree/train/images/loan-oak-tre...,0.5,0.5,1.0,1.0,2
541,../data/DATASET/Tree/train/images/TCC-2021_jpg...,0.886719,0.420312,0.226562,0.840625,2


### 2.3.2. Обработка датасета TreeDetection 

In [16]:
dataTreeDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_DETECTION}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTreeDetection = pd.concat([dataTreeDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataTreeDetection = datasetMinimization(dataTreeDetection, DATASET_DOWNGRADES[print_variable_name(dataTreeDetection)])

dataTreeDetection.sample(10) # Посмотрим на данные

  0%|          | 0/1803 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

There were 2060 photos, now there are 1030


,image_name,x_center,y_center,w,h,classes
2555,../data/DATASET/TreeDetection/train/images/9bf...,0.479687,0.430469,0.402344,0.85625,2
2263,../data/DATASET/TreeDetection/train/images/6da...,0.414844,0.850781,0.4375,0.298438,2
86,../data/DATASET/TreeDetection/train/images/07a...,0.5,0.5,1.0,1.0,2
1009,../data/DATASET/TreeDetection/train/images/220...,0.582812,0.453125,0.81875,0.732812,2
399,../data/DATASET/TreeDetection/train/images/210...,0.427344,0.375781,0.56875,0.221875,2
648,../data/DATASET/TreeDetection/train/images/211...,0.635938,0.413281,0.25625,0.425,2
317,../data/DATASET/TreeDetection/train/images/210...,0.23125,0.486719,0.404687,0.959375,2
1581,../data/DATASET/TreeDetection/train/images/230...,0.725,0.91875,0.35,0.1625,2
87,../data/DATASET/TreeDetection/train/images/091...,0.110937,0.732031,0.221875,0.400781,2
143,../data/DATASET/TreeDetection/train/images/126...,0.432031,0.535937,0.079687,0.311719,2


### 2.3.3. Обработка датасета Tree 100

In [17]:
dataTree100 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_TREE_100}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_TREE_100}{sample}/labels/', 2)) # Получаем датасет с разметкой
    dataTree100 = pd.concat([dataTree100, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

dataTree100.sample(10) # Посмотрим на данные

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

,image_name,x_center,y_center,w,h,classes
94,../data/DATASET/Trees100/train/images/arvore41...,0.601562,0.339844,0.028125,0.074219,2
27,../data/DATASET/Trees100/valid/images/arvore58...,0.515625,0.64375,0.199219,0.225781,2
76,../data/DATASET/Trees100/train/images/arvore34...,0.498437,0.484375,0.635938,0.83125,2
13,../data/DATASET/Trees100/valid/images/arvore12...,0.775781,0.690625,0.192188,0.416406,2
51,../data/DATASET/Trees100/train/images/arvore12...,0.448437,0.250781,0.323437,0.34375,2
112,../data/DATASET/Trees100/train/images/arvore51...,0.78125,0.498437,0.039062,0.159375,2
5,../data/DATASET/Trees100/train/images/arvore03...,0.614062,0.149219,0.094531,0.290625,2
13,../data/DATASET/Trees100/train/images/arvore04...,0.500781,0.799219,0.154688,0.259375,2
29,../data/DATASET/Trees100/valid/images/arvore61...,0.624219,0.795312,0.05,0.076563,2
28,../data/DATASET/Trees100/train/images/arvore11...,0.682031,0.386719,0.453906,0.53125,2


## 2.4. Обработка датасета с птицами
### 2.4.1. Обработка датасета с Bird Annotation

In [18]:
dataBirdAnnotation = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRD_ANNOTATION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRD_ANNOTATION}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdAnnotation  = pd.concat([dataBirdAnnotation, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdAnnotation = datasetMinimization(dataBirdAnnotation, DATASET_DOWNGRADES[print_variable_name(dataBirdAnnotation)])

dataBirdAnnotation.sample(10) # Посмотрим на данные

  0%|          | 0/1362 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

There were 1557 photos, now there are 389


,image_name,x_center,y_center,w,h,classes
5630,../data/DATASET/BirdAnnotation/train/images/pa...,0.298438,0.575,0.107813,0.101562,3
630,../data/DATASET/BirdAnnotation/test/images/vol...,0.476562,0.471875,0.0375,0.0875,3
15348,../data/DATASET/BirdAnnotation/train/images/vo...,0.427344,0.614844,0.05625,0.042188,3
2681,../data/DATASET/BirdAnnotation/train/images/e1...,0.544531,0.464844,0.064062,0.082812,3
3838,../data/DATASET/BirdAnnotation/train/images/e2...,0.203906,0.421094,0.085938,0.085938,3
13938,../data/DATASET/BirdAnnotation/train/images/vo...,0.510156,0.79375,0.270313,0.215625,3
1504,../data/DATASET/BirdAnnotation/train/images/e1...,0.248438,0.482031,0.028125,0.035937,3
1555,../data/DATASET/BirdAnnotation/valid/images/vo...,0.374219,0.539062,0.089063,0.21875,3
14141,../data/DATASET/BirdAnnotation/train/images/vo...,0.45625,0.255469,0.1,0.103125,3
7404,../data/DATASET/BirdAnnotation/train/images/vo...,0.667969,0.335938,0.03125,0.0375,3


### 2.4.2. Обработка датасета с Birds Detection

In [19]:
dataBirdsDetectod = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_DETECTOR}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_DETECTOR}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsDetectod = pd.concat([dataBirdsDetectod, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsDetectod = datasetMinimization(dataBirdsDetectod, DATASET_DOWNGRADES[print_variable_name(dataBirdsDetectod)])

dataBirdsDetectod.sample(10)  # Посмотрим на данные

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

There were 849 photos, now there are 212


,image_name,x_center,y_center,w,h,classes
261,../data/DATASET/BirdsDetector/train/images/bas...,0.385937,0.582031,0.624219,0.835938,3
2626,../data/DATASET/BirdsDetector/train/images/vol...,0.271094,0.721094,0.021875,0.098437,3
967,../data/DATASET/BirdsDetector/valid/images/paj...,0.320312,0.558594,0.014063,0.025,3
1807,../data/DATASET/BirdsDetector/valid/images/vol...,0.355469,0.428125,0.111719,0.097656,3
48,../data/DATASET/BirdsDetector/train/images/274...,0.169531,0.685156,0.335938,0.526563,3
840,../data/DATASET/BirdsDetector/train/images/e11...,0.689844,0.491406,0.033594,0.039062,3
651,../data/DATASET/BirdsDetector/valid/images/e28...,0.545312,0.439844,0.275,0.1125,3
2360,../data/DATASET/BirdsDetector/train/images/vol...,0.521875,0.367188,0.114844,0.036719,3
816,../data/DATASET/BirdsDetector/valid/images/paj...,0.939844,0.188281,0.035937,0.058594,3
375,../data/DATASET/BirdsDetector/test/images/gg10...,0.639844,0.470313,0.076563,0.180469,3


### 2.4.3. Обработка датасета с BirdsV3

In [20]:
dataBirdsV3 = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BIRDS_V3}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BIRDS_V3}{sample}/labels/', 3)) # Получаем датасет с разметкой
    dataBirdsV3 = pd.concat([dataBirdsV3, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBirdsV3 = datasetMinimization(dataBirdsV3, DATASET_DOWNGRADES[print_variable_name(dataBirdsV3)])

dataBirdsV3.sample(10)  # Посмотрим на данные

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

There were 665 photos, now there are 332


,image_name,x_center,y_center,w,h,classes
1885,../data/DATASET/BirdsV3/train/images/birds-roo...,0.561719,0.575781,0.039062,0.04375,3
1046,../data/DATASET/BirdsV3/train/images/83ff5a336...,0.930469,0.378906,0.042188,0.05,3
2147,../data/DATASET/BirdsV3/train/images/birds-roo...,0.236719,0.421094,0.042188,0.071875,3
3599,../data/DATASET/BirdsV3/train/images/Pigeons-o...,0.783594,0.395313,0.135937,0.121875,3
4091,../data/DATASET/BirdsV3/train/images/pigeons-t...,0.886719,0.374219,0.05,0.070312,3
135,../data/DATASET/BirdsV3/train/images/1_jpg.rf....,0.145313,0.792188,0.04375,0.071875,3
616,../data/DATASET/BirdsV3/train/images/454196843...,0.035156,0.516406,0.048438,0.054688,3
2456,../data/DATASET/BirdsV3/train/images/DSC_0597_...,0.317969,0.240625,0.020313,0.028125,3
1704,../data/DATASET/BirdsV3/train/images/birds-on-...,0.413281,0.800781,0.014063,0.034375,3
2635,../data/DATASET/BirdsV3/train/images/four-bird...,0.628906,0.444531,0.03125,0.065625,3


## 2.5. Обработка фотографий со зданиями
### 2.5.1. Обработка датасета Building Detection 

In [21]:
dataBuildingDetection = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_BUILDING_DETECTION}{sample}/labels') # Список файлов с разметкой 
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_BUILDING_DETECTION}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataBuildingDetection = pd.concat([dataBuildingDetection, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataBuildingDetection = datasetMinimization(dataBuildingDetection, DATASET_DOWNGRADES[print_variable_name(dataBuildingDetection)])

dataBuildingDetection.sample(10)  # Посмотрим на данные

  0%|          | 0/726 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

There were 802 photos, now there are 401


,image_name,x_center,y_center,w,h,classes
70,../data/DATASET/BuildingDetection/train/images...,0.519231,0.573317,0.956731,0.848558,4
511,../data/DATASET/BuildingDetection/train/images...,0.512019,0.513221,0.774038,0.96875,4
336,../data/DATASET/BuildingDetection/train/images...,0.5,0.498798,0.995192,0.997596,4
97,../data/DATASET/BuildingDetection/train/images...,0.497068,0.498798,0.994135,0.997596,4
579,../data/DATASET/BuildingDetection/train/images...,0.498798,0.498798,0.997596,0.997596,4
661,../data/DATASET/BuildingDetection/train/images...,0.498798,0.498798,0.997596,0.997596,4
22,../data/DATASET/BuildingDetection/valid/images...,0.625,0.640625,0.609375,0.711538,4
595,../data/DATASET/BuildingDetection/train/images...,0.498798,0.498798,0.997596,0.997596,4
389,../data/DATASET/BuildingDetection/train/images...,0.498798,0.498798,0.997596,0.997596,4
630,../data/DATASET/BuildingDetection/train/images...,0.498798,0.498798,0.997596,0.997596,4


### 2.5.2. Обработка датасета Josh Edits 

In [22]:
dataJoshEdits = pd.DataFrame(columns=dataCarDetection.columns) 

# Проходимся по всем выборкам
for sample in ['train', 'valid', 'test']:
    labels_name = os.listdir(f'{PATH_JOSH_EDITS}{sample}/labels') # Список файлов с разметкой 
        
    labels = pd.DataFrame(converterYoloToDataFrame(labels_name, f'{PATH_JOSH_EDITS}{sample}/labels/', 4)) # Получаем датасет с разметкой
    dataJoshEdits = pd.concat([dataJoshEdits, labels.T]) # Соединяем выборки в единый датасет

del labels_name, labels # Очищаем память

# Уменьшаем размер DataFrame
dataJoshEdits = datasetMinimization(dataJoshEdits, DATASET_DOWNGRADES[print_variable_name(dataJoshEdits)])

dataJoshEdits.sample(10)  # Посмотрим на данные

  0%|          | 0/3036 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

There were 3251 photos, now there are 1625


,image_name,x_center,y_center,w,h,classes
2758,../data/DATASET/JoshEdits/train/images/modelme...,0.885938,0.377344,0.227344,0.152344,4
247,../data/DATASET/JoshEdits/test/images/zombie-h...,0.535937,0.5,0.496094,1.0,4
1886,../data/DATASET/JoshEdits/train/images/image2_...,0.43125,0.520312,0.8625,0.550781,4
469,../data/DATASET/JoshEdits/train/images/37_3903...,0.903125,0.480469,0.19375,0.257812,4
425,../data/DATASET/JoshEdits/train/images/37_3163...,0.5,0.525,1.0,0.384375,4
2116,../data/DATASET/JoshEdits/train/images/image_3...,0.620313,0.614844,0.280469,0.624219,4
903,../data/DATASET/JoshEdits/train/images/b5c0880...,0.513281,0.48125,0.790625,0.729688,4
5219,../data/DATASET/JoshEdits/train/images/sv_42-2...,0.960156,0.417187,0.079687,0.159375,4
3893,../data/DATASET/JoshEdits/train/images/sv_37-3...,0.913281,0.458594,0.150781,0.2,4
5569,../data/DATASET/JoshEdits/train/images/sv_42-3...,0.603906,0.510938,0.178906,0.091406,4


# 3. Объединение датасетов в единый датасет

In [23]:
data = pd.concat(
    [
        dataCarDetection, dataCarPerson, dataCarsDetection, # Авто
        dataSTN, dataUnilityPole, dataTransmission, # ЛЭП
        dataTree, dataTreeDetection, dataTree100, # Деревья
        dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3, # Птицы
        dataBuildingDetection, dataJoshEdits # Здания
    ], 
    ignore_index=True
) # Объеденим датасеты в один единый

del dataCarDetection, dataCarPerson, dataCarsDetection # очищаем память от датасетов с Авто
del dataSTN, dataUnilityPole, dataTransmission # очищаем память от датасетов с ЛЭП
del dataTree, dataTreeDetection, dataTree100 # очищаем память от датасетов со Деревьями
del dataBirdAnnotation, dataBirdsDetectod, dataBirdsV3 # очищаем память от датасетов со Птицами
del dataBuildingDetection, dataJoshEdits # очищаем память от датасетов со Зданиями

data.sample(10) # Посмотрим на данные

,image_name,x_center,y_center,w,h,classes
26564,../data/DATASET/BirdsDetector/train/images/vol...,0.601562,0.396094,0.025781,0.010937,3
30732,../data/DATASET/BuildingDetection/train/images...,0.498798,0.5,0.997596,1.0,4
2924,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.03125,0.674279,0.048077,0.168269,0
16227,../data/DATASET/Transmission/data_original_siz...,0.49987,0.49987,0.99974,0.014226,5
2998,../data/DATASET/CarPerson/Car-Person-v2-Robofl...,0.765625,0.338942,0.038462,0.032452,0
18833,../data/DATASET/TreeDetection/train/images/230...,0.4875,0.5,0.975,1.0,2
18006,../data/DATASET/Tree/train/images/trees-isolat...,0.189062,0.744531,0.378125,0.510938,2
26097,../data/DATASET/BirdsDetector/train/images/e11...,0.361719,0.614062,0.035937,0.042969,3
22628,../data/DATASET/BirdAnnotation/train/images/vo...,0.508594,0.479687,0.029687,0.025,3
15838,../data/DATASET/Transmission/data_original_siz...,0.48151,0.48151,0.278125,0.99966,5


In [24]:
data['image_name'].unique().shape # Количество фотографий

(7105,)

In [25]:
# Изменим тип данных координат с string на float
for column in data.columns[1:-1]:
    data[column] = data[column].astype(float)

data['classes'] = data['classes'].astype(int) # Изменим тип классов с string на int
data.info() # Посмотрим на сведение дата сета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34097 entries, 0 to 34096
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  34097 non-null  object 
 1   x_center    34097 non-null  float64
 2   y_center    34097 non-null  float64
 3   w           34097 non-null  float64
 4   h           34097 non-null  float64
 5   classes     34097 non-null  int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 1.4+ MB


In [26]:
count_image = data['image_name'].unique().shape[0]
print(f'Dataset contains {count_image} unique photos and {data.shape[0]} objects')
del count_image # Очищаем память

Dataset contains 7105 unique photos and 34097 objects


    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [27]:
# Выведем в процентах количество уникальных изображений по классам
data.groupby('image_name')['classes'].head().value_counts().sort_index() /  data.groupby('image_name')['classes'].head().shape[0] * 100

classes
0    21.319879
1     3.933736
2    13.722210
3    15.227714
4    19.938485
5    25.857975
Name: count, dtype: float64

    0 - машина
    1 - ЛЭП
    2 - Деревья
    3 - Птицы
    4 - Дом/здание
    5 - Провода

In [28]:
# Распределение классов среди всех объектов
data['classes'].value_counts().sort_index() / data.shape[0] * 100

classes
0    18.758249
1     2.577939
2     8.120949
3    30.067161
4    10.927648
5    29.548054
Name: count, dtype: float64

In [29]:
# Разделим данные на выборки
train_name_file, test_name_file = train_test_split(data['image_name'].unique(), test_size=.30, random_state=42) 
valid_name_file, test_name_file = train_test_split(test_name_file, test_size=.33, random_state=42)

# Посмотрим на процентное отношение выборок 
pd.DataFrame([len(train_name_file), len(valid_name_file), len(test_name_file)], columns=['size']) / len(data['image_name'].unique()) * 100

,size
0,69.992963
1,20.098522
2,9.908515


In [30]:
train_dataCarDetection = data[data['image_name'].isin(train_name_file)] # Записи для тренировочных данных
valid_dataCarDetection = data[data['image_name'].isin(valid_name_file)] # Записи для валидационных данных
test_dataCarDetection = data[data['image_name'].isin(test_name_file)] # Записи для тестовых данных
del data, train_name_file, valid_name_file, test_name_file # Очистим память

In [31]:
# Название деректорий для создания деректорий
list_paths_to_create = [
    '../data/DATA', '../data/DATA/train', 
    '../data/DATA/valid', '../data/DATA/test', '../data/DATA/train/images', 
    '../data/DATA/train/labels', '../data/DATA/valid/images', '../data/DATA/valid/labels',
    '../data/DATA/test/images', '../data/DATA/test/labels', './yolo_cache'
]

# Создаём папки
for path in list_paths_to_create:
    if not os.path.isdir(path):
        os.mkdir(path)

del list_paths_to_create # Очищаем память

In [32]:
def copyFileDataSet(dataCarDetection, pathEnd):
    '''
        Функция позволяет скопировать фото в новую папку и под каждую из них создать txt файл с разметкой
        Input:
            dataCarDetection - DataFrame содержащий в себе следующиго типа:
                Признаки    Non-Null?  Тип
                image_name  non-null   object 
                x_center    non-null   float64
                y_center    non-null   float64
                w           non-null   float64
                h           non-null   float64
                classes     non-null   int64
            pathEnd - путь созданной папки, внутри которой созданы папки
                images и labels
    '''
    nameFiles = dataCarDetection['image_name'].unique() # Получаем список уникальных файлов

    # Запускаем цикл по все файлам
    for item in tqdm(nameFiles):
        try:
            img = cv2.imread(item, cv2.IMREAD_UNCHANGED)
            img = cv2.resize(cv2.UMat(img), (1280, 720), cv2.INTER_NEAREST)
            # Адаптация путей под Windows
            item = item.replace('\\', '/')
            pathEnd = pathEnd.replace('\\', '/')
            
            cv2.imwrite(pathEnd + '/images/' + item.split('/')[-1], img)
            
            dataList = list(dataCarDetection[dataCarDetection['image_name'] == item].values) # Получаем все объекты на фото
            labelsFile = open(pathEnd + '/labels/' + item.split('/')[-1][:-3] + 'txt', 'w+') # Создаём текстовый документ с разметкой
            isFirst = True # Флаг первой строки
    
            # Запускаем цикл по всем объектам на фото
            for data in dataList:
    
                # Получаем линию с данными 
                line = '' if isFirst else '\n'
                isFirst = False
                line += str(data[-1]) + ' ' + ' '.join(str(number) for number in data[1:-1]) 
                
                labelsFile.write(line) # Записываем в файл
            labelsFile.close() # Закрываем файл
        except:
            pass
        
copyFileDataSet(train_dataCarDetection, '../data/DATA/train') # Формируем тренировочную выборку
copyFileDataSet(valid_dataCarDetection, '../data/DATA/valid') # Формируем валидационную выборку
copyFileDataSet(test_dataCarDetection, '../data/DATA/test') # Формируем тестовую выборку
del train_dataCarDetection, valid_dataCarDetection, test_dataCarDetection # Очищаем память

  0%|          | 0/4973 [00:00<?, ?it/s]

  0%|          | 0/1428 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

In [17]:
# Глобальный путь до директории с проектом 
global_path = Path(os.path.abspath('./TrainNeuralNetwork.ipynb')).parent.parent

yolo_format = dict(
    path=f'{global_path}/data/DATA',
    train='./train',
    val='./valid',
    test='./test',       
    nc=6,
    names={
        0: 'Car',
        1: 'STN',
        2: 'Tree',
        3: 'Bird',
        4: 'House',
        5: 'Cable',
    }
)

del global_path # Очищаем память
             
with open('../data/DATA/yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [2]:
epochTraning = 250 # Количество эпох обучения
pathYalm = "../data/DATA/yolo.yaml" # Путь до файла yalm
patience = 30 # Терпимость эпох для переобучения
imgsz = 1280 # Размер изображения

In [3]:
model = YOLO('yolov8n.yaml').load('./yolo_cache/yolov8n.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz,
    device=0,
    project='./yolo_cache/'
) 

100%|█████████████████████████████████████████████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 9.95MB/s]


Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.47  Python-3.9.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=../data/DATA/yolo.yaml, epochs=250, time=None, patience=30, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=0, workers=8, project=./yolo_cache/, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning D:\MyProjects\Хакатоны\TerraWing\src\data\DATA\train\labels.cache... 4973 images, 873 backgrounds, 0 co

train: WARNING  D:\MyProjects\\TerraWing\src\data\DATA\train\images\image_000001896_jpg.rf.7f980a9461d49d6363444b88bd2695ed.jpg: 1 duplicate labels removed
train: WARNING  D:\MyProjects\\TerraWing\src\data\DATA\train\images\image_000002207_jpg.rf.4ceec290f20f6f33d39b4ad5f715cfc1.jpg: 1 duplicate labels removed



val: Scanning D:\MyProjects\Хакатоны\TerraWing\src\data\DATA\valid\labels... 1428 images, 241 backgrounds, 0 corrupt: 1


val: WARNING  D:\MyProjects\\TerraWing\src\data\DATA\valid\images\image_000001036_jpg.rf.3eb241105e26834305cefe3448dc4f6e.jpg: 1 duplicate labels removed
val: New cache created: D:\MyProjects\\TerraWing\src\data\DATA\valid\labels.cache
Plotting labels to yolo_cache\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to yolo_cache\train8
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/250      17.1G      1.825      4.766      1.839        157       1280:  26%|██▌       | 81/311 [01:11<03:23,  1

KeyboardInterrupt



In [ ]:
model = YOLO('yolov5n6.yaml').load('./yolo_cache/yolov5n6.pt')
model.train(
    data=pathYalm, 
    epochs=epochTraning, 
    patience=patience, 
    imgsz=imgsz,
    device=0,
    project='./yolo_cache/'
)

In [ ]:
def get_metrics():
    """
    Функция позволяет получить данные о метриках всех моделей
    Output:
        metrics - DataFrame содержащий метрики всех моделей
    """
    # Словарь с названиями моделей
    name_model = {
        1: 'YOLOV8n',
        2: 'YOLOV5n',
    }
    
    # DaraFrame для метрик
    metrics = pd.DataFrame() 
    
    # Получаем все метрики по каждой модели
    for i in range(1, 3):
        # Загружаем файл csv в DataFrame
        df = pd.read_csv(f'./yolo_cache/detect/train{i}/results.csv').astype(np.float32)

        # Отбираем метрики
        df = df[[(column) for column in df.columns if 'val' in column or 'metrics' in column and 'metrics/mAP50-95(B)' not in column]]
        df.columns = [f'{name_model[i]}/{column}'.replace(' ', '') for column in df.columns]

        # Соединяем метрики модели с другими
        metrics = pd.concat([metrics, df], axis=1)
        
    return metrics

In [ ]:
# Переименуем папку с первой моделью
if os.path.isdir('./yolo_cache/train/'):
    os.rename('./yolo_cache/train/', './yolo_cache/train1/')

# Создаём DataFrame с данными о метриках 
metrics = get_metrics()

# Создаём рисунок с вложенными графиками для каждого изображения
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Переменная с индексом метрики
index_column = 0

# Пройдёмся по каждому из вложенных графиков
for x in range(0, 2):
    for y in range(0, 3):
        # Добавим на график данные метрики по каждой из моделей
        for column, style in zip(metrics.iloc[:, index_column::6].columns, ['--', '-.']):
            axes[x, y].plot(metrics[column], style)
        
        # Подпишем название графика и его оси
        axes[x, y].legend([column.split('/')[0] for column in metrics.columns[index_column::6]])
        axes[x, y].title.set_text(metrics.columns[index_column].split('/')[2])
        axes[x, y].set_xlabel('Epoch')
        axes[x, y].set_ylabel(metrics.columns[index_column].split('/')[2])
        
        index_column += 1
fig.savefig('./yolo_cache/metrics_version_models.png')